## Modelling for Person A session 1 using all tags

In [ ]:
import numpy as np
from hmmlearn.hmm import GaussianHMM
from itertools import permutations
from scipy.linalg import norm
from hmmlearn import hmm
import math
from datetime import datetime
from numpy import genfromtxt
from sklearn import preprocessing
import hmm_rand_observations_analysis as smp

input_raw = genfromtxt('/Users/Madhu/Documents/HMMHealth_new/A01.txt', delimiter=',',dtype='str')
input = input_raw[:,(2,4,5,6,7)]
walk=np.where(input[:,4]=='walking')[0]
sit=np.where(input[:,4]=='sitting')[0]
lay=np.where(input[:,4]=='lying')[0]
attr=walk.tolist()
attr.extend(sit.tolist())
attr.extend(lay.tolist())
input=input[attr]
encoding = preprocessing.LabelEncoder()  
input[:, 4] = encoding.fit_transform(input[:, 4])
mapping=[]
for i in sorted(set(input[:,4].astype(float))):
    mapping.append((int(i),encoding.inverse_transform(int(i))),)
input = sorted(input, key = lambda a_entry: a_entry[0]) 
input = np.array(input)
input = input[:,(1,2,3,4)]
input = input.astype(float)
train_rows = int(math.floor(0.7* input.shape[0]))
test_rows = input.shape[0] - train_rows    
# separate out training and testing data
trainX = input[:train_rows,:-1]
trainZ = input[:train_rows,-1]
trainZ = trainZ.astype(int)
testX = input[train_rows:,:-1]
testZ = input[train_rows:,-1]
testZ = testZ.astype(int)

In [4]:
model_p = GaussianHMM(3,'diag').fit(trainX)
hidden_states = model_p.predict(trainX)
l = list(permutations(range(0,3)))

In [5]:
acctrain = -1
cur_acc = 0
for p in l:
    var_trainZ = trainZ
    for i in range(trainZ.shape[0]):
        var_trainZ[i] = p[trainZ[i]]
    cur_acc = np.mean( hidden_states == var_trainZ )
    if(cur_acc > acctrain):
        acctrain = cur_acc
        perm_m = p

In [7]:
predZ = model_p.predict(testX)
var_testZ = testZ
for i in range(testZ.shape[0]):
    var_testZ[i] = perm_m[testZ[i]]
acctest = np.mean(predZ == var_testZ)

## Accuracy

In [8]:
print acctrain,acctest

0.614682121531 0.157248157248
